<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Store Sales Forecasting with Prophet using Script Table Operator </b>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.<br>
<br>  
Our Main Objective is to predict sales of store in a week. We are using the python Prophet model and using the Script Table operator of Vantage for forecasting the Store Sales.
<br>
    
<p style = 'font-size:16px;font-family:Arial'><b>STO is a convenient and useful way to use external algorithms along with native Vantage functions to operationalize insights WITHOUT needing to sample data or create data silos for analytics.</b></p>
<p style = 'font-size:16px;font-family:Arial'> The STO is a Database object in the heart of running R and Python in-nodes via the Database. SCRIPT enables invoking the external language interpreters to execute your script files. It does so by running the interpreter in a shell environment invoked upon executing an STO query. Script execution takes place in protected mode, that is, outside the Advanced SQL Engine Database.
Pipelines the database I/O to R or Python scripts executed via Linux command line through STO</p>

<p style = 'font-size:16px;font-family:Arial'><b>Processing steps</b>
<li style = 'font-size:14px;font-family:Arial'>User “installs” their script file on Vantage </li>
    <li style = 'font-size:14px;font-family:Arial'>User executes a SQL query with STO</li>
<li style = 'font-size:14px;font-family:Arial'>STO invokes R or Python, passing the installed script</li>
<li style = 'font-size:14px;font-family:Arial'>Script executes in a Linux “forked” child process</li>
    <li style = 'font-size:14px;font-family:Arial'>Script reads from STDIN and writes to STDOUT</li>
    <li style = 'font-size:14px;font-family:Arial'>Script runs on each unit of parallelism</li>
<li style = 'font-size:14px;font-family:Arial'>Each unit of parallelism returns its own results independently</li>
</p>

<p></p>    
<br>  
<p style = 'font-size:16px;font-family:Arial'>Hence as a data science consultant, we are showcasing the complete approach about how we can make prediction of sales for different stores in advance. We are demonstrating how we can train our models and use them for scoring using the ClearScape Analytics platform. The data we are using is a sample dataset and the results and predictions may not be entirely accurate.
</p>



<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>1. Data</b></p>
<p style = 'font-size:16px;font-family:Arial'>The dataset contains historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.</p>

<p style = 'font-size:16px;font-family:Arial'>Most of the fields are self-explanatory. The following are descriptions for those that aren't.</p>

<li style = 'font-size:14px;font-family:Arial'>Store - a unique Id for each store</li>
<li style = 'font-size:14px;font-family:Arial'>Sales - the turnover for any given day (this is what you are predicting)</li>
<li style = 'font-size:14px;font-family:Arial'>Customers - the number of customers on a given day</li>
<li style = 'font-size:14px;font-family:Arial'>Open - an indicator for whether the store was open: 0 = closed, 1 = open</li>
<li style = 'font-size:14px;font-family:Arial'>StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None</li>
<li style = 'font-size:14px;font-family:Arial'>SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools</li>
<li style = 'font-size:14px;font-family:Arial'>StoreType - differentiates between 4 different store models: a, b, c, d</li>
<li style = 'font-size:14px;font-family:Arial'>Assortment - describes an assortment level: a = basic, b = extra, c = extended</li>
<li style = 'font-size:14px;font-family:Arial'>CompetitionDistance - distance in meters to the nearest competitor store</li>
<li style = 'font-size:14px;font-family:Arial'>CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened</li>
<li style = 'font-size:14px;font-family:Arial'>Promo - indicates whether a store is running a promo on that day</li>
<li style = 'font-size:14px;font-family:Arial'>Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating</li>
<li style = 'font-size:14px;font-family:Arial'>Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2</li>
<li style = 'font-size:14px;font-family:Arial'>PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store.</li>
</p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>2. Start by connecting to the Vantage system.</b></p>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
%%capture
!pip install prophet

<p style = 'font-size:18px;font-family:Arial'><b>**RESTART kernel after installing Prophet.</b><i style = 'font-size:16px;font-family:Arial'>**The simplest way to restart the Kernel is by typing zero zero:  0 0</i></p>
<p style = 'font-size:16px;font-family:Arial'>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed.  If you uncomment those installs, be sure to restart the kernel after executing those lines. </p>

In [ ]:
import getpass
import pandas as pd

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
import os
from prophet import Prophet
import warnings
import itertools

import pickle
import base64

from teradataml import *
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
display.max_rows=5


<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
eng.execute('''SET query_band='DEMO=SalesForecastProphetSTO.ipynb;' UPDATE FOR SESSION; ''')

<hr>
<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>2. Getting Data for This Demo </b>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage.  You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage.  There are two statements in the following cell, and one is commented out.  You may switch which mode you choose by changing the comment string. 


In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_ProphetSTO_cloud');"
 # Takes about 25 seconds
%run -i ../run_procedure.py "call get_data('DEMO_ProphetSTO_local');"
 # Takes about 70 seconds

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>3. Prepare data to do some basic Analysis of the Sales data.</b></p>


In [ ]:
store=DataFrame(in_schema('DEMO_ProphetSTO','Store'))
store  

<p style = 'font-size:16px;font-family:Arial'>The Store dataset contains description of the Stores like, StoreType, distance from the Competition Store and also various Promotion codes and Details.</p>

In [ ]:
sales=DataFrame(in_schema('DEMO_ProphetSTO','Sales_Data'))
sales  

<p style = 'font-size:16px;font-family:Arial'>The Store Sales dataset contains the Store, DayofWeek, Date of Sales , Sales done, Customer involved, SalesOpen is a flag mentioning if the Store is Open or Closed and Promotion Code applied for the Sales.</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>4. Data Analysis and Transformation </b></p>
<p style = 'font-size:18px;font-family:Arial'>In this first section we go through the Sales and store data, handle missing values and create new features for further analysis.</p>
<p style = 'font-size:16px;font-family:Arial'>We check the missing values for the CompetitionDistance column and replace it with the median values.</p>

In [ ]:
from teradataml import SimpleImputeFit, SimpleImputeTransform
fit_obj = SimpleImputeFit(data=store,
                              stats_columns="CompetitionDistance",
                              stats="median")

In [ ]:
obj =  SimpleImputeTransform(data=store,
                                 object=fit_obj.output)
 

In [ ]:
store=obj.result
store

<p style = 'font-size:16px;font-family:Arial'>We join the Store and Sales dataset to get the required columns for our analysis.</p>

In [ ]:
sales_store = sales.merge(right = store, how = "inner", on = "store=store",lsuffix='l', rsuffix='r')
sales_store=sales_store.assign(Store=sales_store.l_Store)
sales_store=sales_store.drop(['l_Store', 'r_Store'], axis=1)
sales_store

In [ ]:
sales_store.shape

<p style = 'font-size:16px;font-family:Arial'>The final dataset used for analysis contains 18 columns and 91,256 rows.</p>
<p style = 'font-size:16px;font-family:Arial'>Based on the data available we do some transformations on the data and create various features. From the SalesDate we, generate columns like , Year, Month, DayOfWeek , WeekofYear etc. Using the columns related to Competition like CompetionOpenSinceYear and CompetitionOpenSinceMonth we calculate if the Competition Store is Open or not(CompetitionOpen). Similarly we do the processing for Promotions and create a flag(PromoOpen)</p>

In [ ]:
# loop for features creation
def features_create(data):
    
    
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)
    
    
    data['Year'] = data.SalesDate.dt.year
    data['Month'] = data.SalesDate.dt.month
    data['Day'] = data.SalesDate.dt.day
    data['DayOfWeek'] = data.SalesDate.dt.dayofweek
    # data['WeekOfYear'] = data.Date.dt.weekofyear
    data['WeekOfYear'] = data['SalesDate'].dt.isocalendar().week
    
    
    
    data['CompetitionOpen'] = 12 * (data.Year - data.CompetitionOpenSinceYear) +         (data.Month - data.CompetitionOpenSinceMonth)
    data['PromoOpen'] = 12 * (data.Year - data.Promo2SinceYear) +         (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
    data['CompetitionOpen'] = data.CompetitionOpen.apply(lambda x: x if x > 0 else 0)        
    data['PromoOpen'] = data.PromoOpen.apply(lambda x: x if x > 0 else 0)
    
    
  
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    data['monthStr'] = data.Month.map(month2str)
    data.loc[data.PromoInterval == 0, 'PromoInterval'] = ''
    data['IsPromoMonth'] = 0
    for interval in data.PromoInterval.unique():
        if interval != 'None':
            for month in interval.split(','):
                data.loc[(data.monthStr == month) & (data.PromoInterval == interval), 'IsPromoMonth'] = 1

    return data

In [ ]:
pd_sales_store= sales_store.to_pandas(all_rows=True)
pd_sales_store["SalesDate"] = pd_sales_store["SalesDate"].astype('datetime64[ns]')
pd_sales_store["PromoInterval"] = pd_sales_store["PromoInterval"].astype('str')
pd_sales_store["StateHoliday"] = pd_sales_store["StateHoliday"].astype('str')
pd_sales_store["StoreType"] = pd_sales_store["StoreType"].astype('str')
pd_sales_store["Assortment"] = pd_sales_store["Assortment"].astype('str')

In [ ]:
pd_sales_store['CompetitionOpenSinceYear'] = pd_sales_store['CompetitionOpenSinceYear'].replace('None', 0)
pd_sales_store['CompetitionOpenSinceMonth'] = pd_sales_store['CompetitionOpenSinceMonth'].replace('None', 0)
pd_sales_store['Promo2SinceYear'] = pd_sales_store['Promo2SinceYear'].replace('None', 0)
pd_sales_store['Promo2SinceWeek'] = pd_sales_store['Promo2SinceWeek'].replace('None', 0)

In [ ]:
pd_sales_store['CompetitionOpenSinceYear'] = pd_sales_store['CompetitionOpenSinceYear'].replace(np.nan, 0)
pd_sales_store['CompetitionOpenSinceMonth'] = pd_sales_store['CompetitionOpenSinceMonth'].replace(np.nan, 0)
pd_sales_store['Promo2SinceYear'] = pd_sales_store['Promo2SinceYear'].replace(np.nan, 0)
pd_sales_store['Promo2SinceWeek'] = pd_sales_store['Promo2SinceWeek'].replace(np.nan, 0)

In [ ]:
pd_sales_store.head(5)

In [ ]:
features_pd=features_create(pd_sales_store)
features_pd.head(5)

In [ ]:
sns.barplot(x = 'Month', y = 'Sales', data = features_pd)
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The above graph shows the total sales across months for all stores. We can see that the sales are highest in December which is the Holiday Season.</p>
<p style = 'font-size:16px;font-family:Arial'>Now we will see the same metrics across different Store types and also based on whether there was any Promotion available(Promo=1) or not (Promo=0)

In [ ]:
#Catplot month Vs Sales
sns.catplot(data = features_pd, x = 'Month', y = "Sales", 
               col = 'StoreType', # per store type in cols
               palette = 'plasma',
               hue = 'StoreType',
               row = 'Promo', # per promo in the store in rows
               color = 'Year') 

<p style = 'font-size:16px;font-family:Arial'>All store types follow the same trend but at different scales depending on the presence of the promotion `Promo` and `StoreType` except for the StoreType = b.
</p>


<p style = 'font-size:16px;font-family:Arial'>The above graph shows the Sales per Month. We can see that there are peaks mainly during the Year end period.</p>
<p style = 'font-size:16px;font-family:Arial'>Next we try to get four stores from store types to represent their group**:</p>
<li style = 'font-size:16px;font-family:Arial'> Store number 2 for `StoreType` A</li>
<li style = 'font-size:16px;font-family:Arial'>Store number 25 for `StoreType` B</li>
<li style = 'font-size:16px;font-family:Arial'>Store number 1 for `StoreType` C</li> 
<li style = 'font-size:16px;font-family:Arial'>Store number 13 for `StoreType` D</li>

<p style = 'font-size:16px;font-family:Arial'>It also makes sense to downsample the data from days to weeks using the `resample` method to see the present trends more clearly.</p>

In [ ]:
# Sorting with 'date'
train_df = pd_sales_store.set_index('SalesDate')
# Sales datacheck
train_df['Sales'] = train_df['Sales'] * 1.0
# storewise sales data
sales_a = train_df[train_df.Store == 2]['Sales']
sales_b = train_df[train_df.Store == 85]['Sales'].sort_index(ascending = True) # solve the reverse order
sales_c = train_df[train_df.Store == 1]['Sales']
sales_d = train_df[train_df.Store == 13]['Sales']

In [ ]:
f, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (12, 13))

# store types
sales_a.resample('W').mean().plot(ax = ax1)
sales_b.resample('W').sum().plot(ax = ax2)
sales_c.resample('W').sum().plot(ax = ax3)
sales_d.resample('W').sum().plot(ax = ax4)

<p style = 'font-size:16px;font-family:Arial'>Retail sales for `StoreType` A and C tend to peak for the Christmas season and then decline after the holidays. We might have seen the same trend for `StoreType` D (at the bottom) but there is no information from July 2014 to January 2015 about these stores as they were closed.</p>

<p style = 'font-size:16px;font-family:Arial'>Next we check the Yearly trend for these Store Types thing to check the presence of a trend in series.</p>

In [ ]:
# Seasonal decompose
from statsmodels.tsa.seasonal import seasonal_decompose

f, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (15, 15))

# monthly
decomposition_a = seasonal_decompose(sales_a, model = 'additive', extrapolate_trend='freq', period=1)
decomposition_a.trend.plot(ax = ax1)

decomposition_b = seasonal_decompose(sales_b, model = 'additive', extrapolate_trend='freq', period=1)
decomposition_b.trend.plot( ax = ax2)

decomposition_c = seasonal_decompose(sales_c, model = 'additive', extrapolate_trend='freq', period=1)
decomposition_c.trend.plot( ax = ax3)

decomposition_d = seasonal_decompose(sales_d, model = 'additive', extrapolate_trend='freq', period=1)
decomposition_d.trend.plot( ax = ax4)

<p style = 'font-size:16px;font-family:Arial'>Overall sales seems to increase, however not for the `StoreType` C (a third from the top). Even though the `StoreType` A is the most selling store type in the dataset, it seems that it follows the same decreasing trajectory as `StoreType` C did.</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5. Checking for Stationarity of Time Series using the Dickey Fuller Test</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here we will check for stationarity of all time series using the Dickey-Fuller Test. For more info on the test,  see <a href="https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-VantageTM-Unbounded-Array-Framework-Time-Series-Reference-17.20/Diagnostic-Statistical-Test-Functions/TD_DICKEY_FULLER/TD_DICKEY_FULLER-Example">here.</a> 
<p style = 'font-size:16px;font-family:Arial'>The null hypothesis for the test is that the data is non-stationary. We want to REJECT the null hypothesis for this test. So we want a p-value of less that 0.05 (or smaller) and a negative coefficient value for the lag term in our regression model.</p> 
<p style = 'font-size:16px;font-family:Arial'>The Dickey fuller function needs series data so we use the TDSeries function to create a series and apply DickeyFuller to check the stationarity of the data.</p>


In [ ]:
from teradataml import DickeyFuller
data_series_df = TDSeries(data=sales_store,
                              id="SalesDate",
                              row_index=("Store"),
                              row_index_style= "SEQUENCE",
                              payload_field="Sales",
                              payload_content="REAL")

In [ ]:
df_out = DickeyFuller(   data=data_series_df,
                           algorithm='NONE')

# Print the result DataFrame.
print(df_out.result)

<p></p>
<p style = 'font-size:16px;font-family:Arial'>In the above output the p-value corresponding to the calculated test statistic is less than 0.05. It means that the series is stationary. The output column NULL_HYP which means NULL HYPOTHESIS can have 2 values 
    <li style = 'font-size:16px;font-family:Arial'>ACCEPT means the null hypothesis is accepted. No Unit roots are present, and therefore the process is stationary.</li>
<li style = 'font-size:16px;font-family:Arial'>REJECT means the null hypothesis is rejected. Unit roots are present, and the process may or may not be stationary, depending on other factors.</li>
</p>
<p style = 'font-size:16px;font-family:Arial'>Since the P Value for the series is less than 0.05 in all cases we consider it as stationary.</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6. Creating the model and forecasting using Prophet and STO.</b></p>

<p style = 'font-size:16px;font-family:Arial'>Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.</p>
<p style = 'font-size:16px;font-family:Arial'>Prophet follows the sklearn model API. We create an instance of the Prophet class and then call its fit and predict methods.</p>


<p style = 'font-size:16px;font-family:Arial'>All the below steps which include the Prophet model are executed in the python in the file <a href="./stoSalesForecastnew.py">stoSalesForecastnew.py</a> file. We then use this py file in the STO command and get the forecasted values.</p>
<p style = 'font-size:16px;font-family:Arial'>The input to Prophet is always a dataframe with two columns: ds and y. The ds (datestamp) column should be of a format expected by Pandas, ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp. The y column must be numeric, and represents the measurement we wish to forecast.</p>

<p style = 'font-size:16px;font-family:Arial'>The below code shows the creation of the Sales DataFrame and the holidays Dataframe which are used in the model creation and model fit.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Create Sales data dataframe using data from Vantage</b></p>

```python 
# create Sales data 
sales = pd_sales_store.rename(columns = {'SalesDate': 'ds','Sales': 'y'})
```  

<p style = 'font-size:16px;font-family:Arial'><b>Create holidays dataframe</b></p>

```python
#create holidays dataframe
     

school_dates = df[df.SchoolHoliday == 1].loc[:, 'Date'].values

school = pd.DataFrame({'holiday': 'school_holiday',
                      'ds': pd.to_datetime(school_dates)})

holidays = school      
```

<p style = 'font-size:16px;font-family:Arial'>We fit the model by instantiating a new Prophet object. Any settings to the forecasting procedure are passed into the constructor. Then you call its fit method and pass in the historical dataframe(sales).</p>

<p style = 'font-size:16px;font-family:Arial'><b>Instantiate and fit model using Prophet</b></p>

```python

# Prophet implementation 
my_model = Prophet(interval_width = 0.95, 
                   holidays = holidays.head(50000))
my_model.fit(sales)                   
```

<p style = 'font-size:16px;font-family:Arial'>Predictions are then made on a dataframe with a column ds containing the dates for which a prediction is to be made. You can get a suitable dataframe that extends into the future a specified number of days using the helper method Prophet.make_future_dataframe. By default it will also include the dates from the history, so we will see the model fit as well.<p>

<p style = 'font-size:16px;font-family:Arial'><b>Create future dates for forecasting</b></p>

```python
dt = min(sales['ds'].values)
date1 = datetime.datetime.strptime(dt, "%y/%m/%d").date()



#  # Subtract one month
start_date = date1 - relativedelta(months=1)

# Get man date and then get future dates for 1 month
dt1 = max(sales['ds'].values)
date2 = datetime.datetime.strptime(dt1, "%y/%m/%d").date()
# date2 = datetime.datetime.strptime(datetime_str, "%Y/%m/%dT%H:%M:%S.%f").date()
end_date = date2 + relativedelta(months=1)
# end_date= str(end_value)


# # date_range = pd.date_range(start_date, periods=num_days)
date_range = pd.date_range(str(start_date), str(end_date))

future_dates = pd.DataFrame({'ds': date_range})               
```

<p style = 'font-size:16px;font-family:Arial'>The predict method will assign each row in future a predicted value which it names yhat. If you pass in historical dates, it will provide an in-sample fit. The forecast object here is a new dataframe that includes a column yhat with the forecast, as well as columns for components and uncertainty intervals.<p>

<p style = 'font-size:16px;font-family:Arial'><b>Create dataframe with forecast values</b></p>

```python
# forecast
forecast = my_model.predict(future_dates.head(10000))               
```

<p style = 'font-size:16px;font-family:Arial'>The forecasted values will be fetched back to Vantage using the Returns clause of the STO as seen in the query below.<p>
<p style = 'font-size:16px;font-family:Arial'>The SCRIPT COMMAND requires the below elements 
<li style = 'font-size:16px;font-family:Arial'><b>ON clause: </b> The SCRIPT function can have only one ON clause (single input). The ON clause can be specified with no options or with: HASH BY, PARTITION BY, PARTITION BY ANY, an optional ORDER BY or LOCAL ORDER BY clause</li>
<li style = 'font-size:16px;font-family:Arial'><b>SCRIPT_COMMAND: </b>
The script to be executed. The SCRIPT_COMMAND is a required keyword.
    <li style = 'font-size:16px;font-family:Arial'><b>runtime_literal_command: </b>The parameters to SCRIPT_COMMAND can be an executable name followed by the script name and other inputs, or any valid LINUX command.</li>
<li style = 'font-size:16px;font-family:Arial'><b>RETURNS: </b>
    The names and types of the output columns returned by the script. <b>* </b>Specifies that all columns of the input table should be returned by the SCRIPT function.</li>
    
<p style = 'font-size:16px;font-family:Arial'>First we will create a dataset which can be passed to the Script Table Operator(STO).</p>    

In [ ]:
qry='''CREATE SET TABLE DEMO_USER.Store_Sales_ID 
     (
      SlsID INTEGER,
      Store INTEGER,
      DayOfWeek INTEGER,
      SalesDate DATE FORMAT 'yyyy/mm/dd',
      Sales INTEGER,
      Customers INTEGER,
      SalesOpen INTEGER,
      Promo INTEGER,
      StateHoliday CHAR(1) CHARACTER SET LATIN NOT CASESPECIFIC,
      SchoolHoliday INTEGER)
      PRIMARY INDEX ( SlsID ); '''
qry1='''insert into DEMO_USER.Store_Sales_ID select 1,  Store ,
      DayOfWeek ,
      SalesDate ,
      Sales ,
      Customers ,
      SalesOpen ,
      Promo ,
      StateHoliday,
      SchoolHoliday  from DEMO_prophetSTO.Sales_Data where Store <= 5;'''
try:
    eng.execute(qry)
    eng.execute(qry1) 
except:
    eng.execute('Drop table DEMO_USER.Store_Sales_ID;')
    eng.execute(qry)
    eng.execute(qry1) 

<p style = 'font-size:16px;font-family:Arial'>First we set the Database for execution to the user database: demo_user in this case</p>

In [ ]:
eng.execute("DATABASE demo_user;")

<p style = 'font-size:16px;font-family:Arial'>Set the search path to the database where the file is installed</p>

In [ ]:
eng.execute("SET SESSION SEARCHUIFDBPATH = demo_user;")

<p style = 'font-size:16px;font-family:Arial'>Install the user script file on Vantage. In case of rerun if the file already exists we first remove it and then install again.</p>

In [ ]:
try:
    eng.execute("call SYSUIF.INSTALL_FILE('stoSalesForecastnew', 'stoSalesForecastnew.py', 'cz!./stoSalesForecastnew.py');")
except:    
    eng.execute("call SYSUIF.REMOVE_FILE('stoSalesForecastnew', 1);")
    eng.execute("call SYSUIF.INSTALL_FILE('stoSalesForecastnew', 'stoSalesForecastnew.py', 'cz!./stoSalesForecastnew.py');")

<p style = 'font-size:16px;font-family:Arial'>Execute the script in SQL using SCRIPT table operator with the following SQL code:</p>

In [ ]:
qry= '''create multiset table STO_OUTPUT as ( SELECT * 
        FROM SCRIPT (ON (SELECT * FROM DEMO_USER.Store_Sales_ID )
        SCRIPT_COMMAND('/opt/teradata/languages/Python/bin/python3 ./demo_user/stoSalesForecastnew.py') 
        RETURNS ('ds timestamp(0), yhat FLOAT, yhat_lower FLOAT, yhat_upper FLOAT , trend FLOAT, weekly FLOAT, yearly FLOAT')) AS d) with data;'''

try:
    eng.execute(qry);
except:
    eng.execute('drop table STO_OUTPUT');
    eng.execute(qry);
# DEMO_ProphetSTO.Sales_Data
# ds timestamp(0), yhat FLOAT, yhat_lower FLOAT, yhat_upper FLOAT , trend FLOAT, weekly FLOAT, yearly FLOAT

In [ ]:
STO_out = DataFrame(in_schema('demo_user','STO_OUTPUT'))
STO_out

In [ ]:
STO_out.shape

<p style = 'font-size:16px;font-family:Arial'>The forecasting output contains information for:
</p>
<li style = 'font-size:16px;font-family:Arial'>The forecasted value (yhat)</li>
<li style = 'font-size:16px;font-family:Arial'>Range for the forecasted values (yhat_lower and yhat_upper)</li>
<li style = 'font-size:16px;font-family:Arial'>The overall trend for a given date (also incorporates seasonality)</li>
<li style = 'font-size:16px;font-family:Arial'>Additive terms to adjust the trend to get the forecasted value</li>

In [ ]:
plot_output = STO_out.to_pandas(all_rows=True).reset_index()
plot_output["ds"] = pd.to_datetime(plot_output['ds']).dt.date

In [ ]:
plot_output_forecast = plot_output[['ds','yhat','yhat_lower','yhat_upper']].sort_values('ds', ascending=True)
# .tail(300)
plot_output_forecast = plot_output_forecast.reset_index()
plot_output_forecast.head(5)

In [ ]:
import matplotlib.pyplot as plt



# Create the data for the line graph, including the x-values and the corresponding upper and lower bounds
x_values = plot_output_forecast['ds'].values
y_values = plot_output_forecast['yhat'].values
lower_bounds = plot_output_forecast['yhat_lower'].values
upper_bounds = plot_output_forecast['yhat_upper'].values

 
plt.figure(figsize=(12, 8))
# Plot the line graph
plt.plot(x_values, y_values, color='black', label='Forecast Values')
plt.fill_between(x_values, lower_bounds, upper_bounds, color='lightblue', alpha=0.3, label='Confidence Interval')

 

# Customize the plot

plt.xlabel('Date')
plt.ylabel('Forecast Values')
plt.title('Forecast Sales Values with Confidence Interval')
plt.legend()

 

# Show the plot
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The above graph contains the Forecast values(black line) and the light blue area is the range of the lower(yhat_lower) and upper(yhat_upper) limits of the forecasted values.</p>

<p style = 'font-size:16px;font-family:Arial'>Below we will check the general trend , the Weekly trend and the yearly trend of the forecasted values</p>

In [ ]:
plot_trend = plot_output[['ds','trend','weekly','yearly']].sort_values('ds', ascending=True)

In [ ]:
import matplotlib.pyplot as plt

# Create subplots
# Create subplots
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 10))

ds = np.array(plot_trend['ds'])
# day_week= np.array(pd.to_datetime(plot_trend['ds']).dt.dayofweek)
trend = np.array(plot_trend['trend'])
# Plot 'trend' column
axes[0].plot(ds,trend)
axes[0].set_xlabel('Date')
axes[0].set_ylabel('trend')

# Plot 'weekly' column
weekly = np.array(plot_trend['weekly'])
axes[1].plot(ds, weekly)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('weekly')

# Plot 'yearly' column
yearly = np.array(plot_trend['yearly'])
axes[2].plot(ds, yearly)
axes[2].set_xlabel('Date')
axes[2].set_ylabel('yearly')

# Adjust layout and spacing
plt.tight_layout()

# Display the plot
plt.show()


<p style = 'font-size:16px;font-family:Arial'>The first plot shows that the sales has been linearly decreasing over time. The second plot highlights the fact that the weekly volume of sales is similar for all weeks, while the third plot shows that the most busy season occurs during the winter holidays and summer holidays.

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Conclusion:</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have trained and validated the Prophet model using the python script and used the Script Table Operator(STO) using data from Vantage. We get the forecasted data in Vantage using the python script.</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>7. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>

In [ ]:
db_drop_table(table_name='Store_Sales_ID', schema_name='demo_user')

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_ProphetSTO');" 
#Takes 20 seconds

In [ ]:
remove_context()

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>